In [ ]:
import tensorflow as tf
from tensorflow import keras
import mlflow
import numpy as np
import io
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from utils import *

In [ ]:
def custom_loss(fk_weight, fk_loss_type, exp_scale=1.0, huber_delta=0.01):
    def loss_fn(y_true, y_pred):
        # Mean Squared Error for joint angles
        angle_loss = tf.keras.losses.Huber(delta=huber_delta)(y_true, y_pred)
        
        # Forward kinematics loss
        fk_true = forward_kinematics_tf(y_true)
        fk_pred = forward_kinematics_tf(y_pred)
        
        if fk_loss_type == 'exponential':
            fk_diff = tf.reduce_sum(tf.square(fk_true - fk_pred), axis=-1)
            fk_loss = tf.reduce_mean(1 - tf.exp(-exp_scale * fk_diff))
        elif fk_loss_type == 'huber':
            fk_loss = tf.keras.losses.Huber(delta=huber_delta)(fk_true, fk_pred)
        elif fk_loss_type == 'linear':
            fk_loss = tf.reduce_mean(tf.abs(fk_true - fk_pred))
        elif fk_loss_type == 'mae':
            fk_loss = tf.keras.losses.mae(fk_true, fk_pred)
        else:
            raise ValueError(f"Unsupported forward kinematics loss type: {fk_loss_type}")
        
        # Combine losses
        total_loss = angle_loss + fk_weight * fk_loss
        return total_loss
    return loss_fn

def create_model(config):
    model = keras.Sequential([
        keras.layers.Dense(128, input_shape=(3,)),
        keras.layers.Activation('relu'),
        keras.layers.Dense(256),
        keras.layers.Activation('relu'),
        keras.layers.Dense(256),
        keras.layers.Activation('relu'),
        keras.layers.Dense(128),
        keras.layers.Activation('relu'),
        keras.layers.Dense(3, activation='tanh')
    ])
    
    model.compile(optimizer='adam', loss=custom_loss(
        config['fk_weight'], 
        config['fk_loss_type'],
        exp_scale=config.get('exp_scale', 1.0),
        huber_delta=config.get('huber_delta', 1.0)
    ))
    return model

def run_single_experiment(config):
    (train_inputs, train_outputs), (test_inputs, test_outputs), input_mean, input_std = load_and_preprocess_data()
    
    valid_inputs, test_inputs, valid_outputs, test_outputs = train_test_split(
        test_inputs, test_outputs, test_size=0.5, random_state=42
    )
    
    mlflow.set_experiment(config['experiment_name'])
    
    with mlflow.start_run(run_name=config['model_name']):
        mlflow.log_params(config)
        
        steps_per_epoch = len(train_inputs) // config['batch_size']
        total_steps = steps_per_epoch * config['epochs']
        warmup_steps = int(0.1 * total_steps)
        
        model = create_model(config)
        
        model_summary = io.StringIO()
        model.summary(print_fn=lambda x: model_summary.write(x + '\n'))
        mlflow.log_text(model_summary.getvalue(), "model_summary.txt")
        
        lr_scheduler = CosineDecayWithWarmupCallback(
            config['initial_learning_rate'],
            warmup_steps,
            total_steps
        )
        verbose_logging = VerboseLoggingCallback()
        lr_logger = LearningRateLogger()
        
        callbacks = [lr_scheduler, verbose_logging, lr_logger]
        
        callback_names = [callback.__class__.__name__ for callback in callbacks]
        mlflow.log_param("callbacks", ", ".join(callback_names))
        
        start_time = time.time()
        history = model.fit(
            train_inputs, train_outputs,
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            validation_data=(valid_inputs, valid_outputs),
            callbacks=callbacks
        )
        training_time = time.time() - start_time
        
        for epoch, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
            mlflow.log_metric("train_loss", loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
        
        mlflow.log_metric("training_time", training_time)
        
        mlflow.tensorflow.log_model(model, "model")
        
        mlflow.log_text(verbose_logging.get_output(), "training_output.txt")
        
        errors, true_xyz, predicted_xyz = evaluate_model(model, test_inputs, test_outputs, input_mean, input_std)
        
        mlflow.log_metric("mean_error", np.mean(errors))
        mlflow.log_metric("median_error", np.median(errors))
        mlflow.log_metric("90th_percentile_error", np.percentile(errors, 90))
        mlflow.log_metric("max_error", np.max(errors))
        
        true_vs_pred_plot_path = f"./Figures/Regularization/{config['model_name']}_true_vs_predicted.png"
        plot_true_vs_predicted(true_xyz, predicted_xyz, f"{config['model_name']} Model: True vs Predicted", save_path=true_vs_pred_plot_path)
        mlflow.log_artifact(true_vs_pred_plot_path)
        
        error_dist_plot_path = f"./Figures/Regularization/{config['model_name']}_error_distribution.png"
        plot_error_distribution(errors, f"{config['model_name']} Model: Error Distribution", save_path=error_dist_plot_path)
        mlflow.log_artifact(error_dist_plot_path)
        
        print(f"\n{config['model_name']} Model:")
        print(f"Mean Error: {np.mean(errors):.4f}")
        print(f"Median Error: {np.median(errors):.4f}")
        print(f"90th Percentile Error: {np.percentile(errors, 90):.4f}")
        print(f"Max Error: {np.max(errors):.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        
        return {
            'model': model,
            'history': history,
            'errors': errors,
            'true_xyz': true_xyz,
            'predicted_xyz': predicted_xyz,
            'training_time': training_time
        }

In [ ]:
# Create configurations
configs = []
fk_weights = [10, 20]
fk_loss_types = ['exponential', 'huber', 'linear', 'mae']
learning_rates = [0.001, 0.01, 0.05]

# Parameters specific to certain loss types
exp_scales = [0.1, 0.5, 2.0]
huber_deltas = [0.01, 0.05]

for fk_weight in fk_weights:
    for fk_loss_type in fk_loss_types:
        for learning_rate in learning_rates:
            if fk_loss_type == 'exponential':
                for exp_scale in exp_scales:
                    config = {
                        "model_name": f"Model_FKWeight_{fk_weight}_ExpLoss_Scale_{exp_scale}_LR_{learning_rate}",
                        "fk_weight": fk_weight,
                        "fk_loss_type": fk_loss_type,
                        "exp_scale": exp_scale,
                        "epochs": 100,
                        "initial_learning_rate": learning_rate,
                        "batch_size": 65536*2,
                        "experiment_name": "Inverse Kinematics Multi-Loss"
                    }
                    configs.append(config)
            elif fk_loss_type == 'huber':
                for huber_delta in huber_deltas:
                    config = {
                        "model_name": f"Model_FKWeight_{fk_weight}_HuberLoss_Delta_{huber_delta}_LR_{learning_rate}",
                        "fk_weight": fk_weight,
                        "fk_loss_type": fk_loss_type,
                        "huber_delta": huber_delta,
                        "epochs": 100,
                        "initial_learning_rate": learning_rate,
                        "batch_size": 65536*2,
                        "experiment_name": "Inverse Kinematics Multi-Loss"
                    }
                    configs.append(config)
            else:  # linear and mae
                config = {
                    "model_name": f"Model_FKWeight_{fk_weight}_{fk_loss_type.capitalize()}Loss_LR_{learning_rate}",
                    "fk_weight": fk_weight,
                    "fk_loss_type": fk_loss_type,
                    "epochs": 100,
                    "initial_learning_rate": learning_rate,
                    "batch_size": 65536*2,
                    "experiment_name": "Inverse Kinematics Multi-Loss"
                }
                configs.append(config)

# Print the total number of configurations
print(f"Total number of configurations: {len(configs)}")

In [ ]:
# Run experiments
for config in configs:
    run_single_experiment(config)